In [19]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from urllib.parse import quote

In [20]:
# 入参 页码
def get_page(page):
    info = '爬取第{index}页'.format(index = page)
    print(info)
    
    try:
        url = 'https://s.taobao.com/search?q=' + quote(KEYWORD)
        browser.get(url)
        if page > 1:
            #等待定位页码
            input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#mainsrp-pager div.form> input')))
            #等待定位提交
            submit = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#mainsrp-pager div.form> span.btn.J_Submit')))
            #输入页码
            input.clear()
            input.send_keys(page)
            #跳转
            submit.click()
        #页码匹配
        wait.until(EC.text_to_be_present_in_element((By.CSS_SELECTOR, '#mainsrp-pager li.item.active> span'), str(page)))
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.m-itemlist .items .item')))
        get_products()
    except TimeoutException:
        get_page(page)

In [21]:
from pyquery import PyQuery as pq
def get_products():
    html = browser.page_source
    doc = pq(html)
    items = doc('#mainsrp-itemlist .items .item').items()
    for item in items:
        product = {'image': item.find('.pic .img').attr('data-src'),
            'price': item.find('.price').text(),
            'deal': item.find('.deal-cnt').text(),
            'title': item.find('.title').text(),
            'shop': item.find('.shop').text(),
            'location': item.find('.location').text()}
        print(product)
        save_to_mongo(product)

In [22]:
import pymongo
MONGO_URL = 'localhost'
MONGO_DB = 'taobao'
MONGO_COLLECTION = 'products'
#连接mongodb
client = pymongo.MongoClient(MONGO_URL)
#连接数据库
db = client[MONGO_DB]
def save_to_mongo(result):
    try:
        if db[MONGO_COLLECTION].insert_one(result):
            print(' 存储到 MongoDB 成功 ')
    except Exception:
        print(' 存储到 MongoDB 失败 ')

In [23]:
browser = webdriver.Chrome()
wait = WebDriverWait(browser, 10)
KEYWORD = 'iPad'

当前问题是，爬取页数过多时，taobao反爬虫机制，自动检测，跳出登陆页面

In [24]:
for i in range(1,6):
    get_page(i)

爬取第1页
{'image': '//g-search1.alicdn.com/img/bao/uploaded/i4/imgextra/i2/15046875/O1CN01Zztxfj20enm5Fg1yK_!!0-saturn_solar.jpg', 'price': '¥ 2898.00', 'deal': '300+人付款', 'title': 'Apple/苹果2021新款 iPad mini6 迷你6平板电脑ipad mini6(第六代)', 'shop': '健康国际营养', 'location': '广东 深圳'}
 存储到 MongoDB 成功 
{'image': '//g-search1.alicdn.com/img/bao/uploaded/i4/imgextra/i3/116538412/O1CN017ViyfX2C0kXN0pIDl_!!0-saturn_solar.jpg', 'price': '¥ 1997.00', 'deal': '100+人付款', 'title': 'Apple/苹果 iPad mini (第六代) 2021新款 iPad mini6迷你平板电脑', 'shop': 'abileutomo', 'location': '广东 深圳'}
 存储到 MongoDB 成功 
{'image': '//g-search1.alicdn.com/img/bao/uploaded/i4/imgextra/i2/12028972/O1CN01fSytmF2G9ELowcIWH_!!0-saturn_solar.jpg', 'price': '¥ 2158.00', 'deal': '800+人付款', 'title': '2022款Apple/苹果 10.9 英寸 iPad (第十代) 平板电脑ipad10代9代', 'shop': '深圳_恒波', 'location': '广东 深圳'}
 存储到 MongoDB 成功 
{'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i1/1917047079/O1CN01eP4Hny22AEbgnH1pF_!!2-item_pic.png', 'price': '¥ 4799.00', 'deal': '', 'title'

爬取第2页
爬取第2页


KeyboardInterrupt: 